In [1]:
import psycopg2
import pandas as pd
from config import datenbank
# Verbindung zur Datenbank herstellen
conn = psycopg2.connect(
            host=datenbank.HOSTNAME,
            dbname=datenbank.DBNAME,
            user=datenbank.USER,
            password=datenbank.PASSWORD,
            port=datenbank.PORT
        )

# SQL-Abfrage
sql_query = 'SELECT * FROM sensebox'

# Daten in ein Pandas DataFrame laden
df = pd.read_sql_query(sql_query, conn)

# Daten in eine CSV-Datei exportieren
df.to_csv('data/sensor_data.csv', index=False)

# Verbindung schließen
conn.close()


/var/folders/x6/2n7zgvqn5f97hpgxds31xvzm0000gn/T/ipykernel_3078/1887983417.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [19]:
import pandas as pd
import psycopg2
import sqlalchemy
from config import datenbank

def get_data_from_postgres():
    """
    Stellt Verbindung zur Datenbank her, ladet und verarbeitet die Daten
    
    Returns:
        DataFrame mit den interpolierten Daten
    """
    # SQLAlchemy engine erstellen
    engine = sqlalchemy.create_engine(f"postgresql://{datenbank.USER}:{datenbank.PASSWORD}@{datenbank.HOSTNAME}:{datenbank.PORT}/{datenbank.DBNAME}")
    try:
        sql_query = "SELECT * FROM sensebox"
        df = pd.read_sql(sql_query, engine)
        df['createdat'] = pd.to_datetime(df['createdat'])
        df = df.sort_values(by='createdat')
        df = df.set_index('createdat').interpolate(method='time').reset_index()
        return df
    except Exception as e:
        print("Fehler beim Laden der Daten aus der Datenbank:", e)
        return None

# Hauptlogik
df = get_data_from_postgres()
if df is None:
    # Laden der CSV-Datei, falls keine Verbindung zur Datenbank möglich ist
    df = pd.read_csv('data/sensor_data.csv')
    df['createdat'] = pd.to_datetime(df['createdat'])  # Ensure 'createdAt' is in datetime format
    df = df.sort_values(by='createdat')
    df = df.set_index('createdat').interpolate(method='time').reset_index()

# Anzeige des DataFrames
df


Fehler beim Laden der Daten aus der Datenbank: (psycopg2.OperationalError) connection to server at "35.192.180.193", port 5432 failed: Network is unreachable
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)


,createdat,luftdruck,temperatur,uvintensität,relluftfeuchte
0,2024-05-20 17:45:00+00:00,NaN,NaN,73.12,53.09
1,2024-05-20 17:46:00+00:00,NaN,NaN,73.12,53.09
2,2024-05-20 17:47:00+00:00,994.190,22.09,73.12,53.19
3,2024-05-20 17:48:00+00:00,994.185,22.06,73.12,52.99
4,2024-05-20 17:49:00+00:00,994.180,22.02,73.12,52.89
...,...,...,...,...,...
43618,2024-06-20 08:06:00+00:00,1004.280,23.09,241.88,69.53
43619,2024-06-20 08:07:00+00:00,1004.260,23.28,241.88,68.83
43620,2024-06-20 08:08:00+00:00,1004.270,23.35,258.75,68.40
43621,2024-06-20 08:09:00+00:00,1004.280,23.50,253.12,68.00
